In [71]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
import requests
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from nltk.corpus import stopwords, reuters
import hvplot.pandas
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/samuelarciniega/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [72]:
load_dotenv()

True

In [73]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [74]:
api_key = os.getenv("news_api")
#print(api_key)

In [75]:
newsapi = NewsApiClient(api_key=api_key)

In [76]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
end_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["GOOGL"]
apple_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=end_date,
    end=start_date,
    after=None,
    until=None,
).df
apple_ticker.dropna(inplace=True)
apple_ticker.head(50)

GOOGL                                         
                              open       high        low     close   volume
time                                                                       
2021-06-08 00:00:00-04:00  2412.00  2419.5203  2394.7800  2397.950  1025802
2021-06-09 00:00:00-04:00  2413.49  2412.8800  2396.9000  2407.840   749436
2021-06-10 00:00:00-04:00  2406.92  2436.0000  2403.7800  2435.150  1124766
2021-06-11 00:00:00-04:00  2439.00  2441.7200  2418.9600  2430.290   884578
2021-06-14 00:00:00-04:00  2432.50  2448.9200  2418.0000  2448.920   865732
2021-06-15 00:00:00-04:00  2448.00  2455.5100  2421.1200  2429.160   800654
2021-06-16 00:00:00-04:00  2437.07  2437.7200  2389.4000  2415.400  1132794
2021-06-17 00:00:00-04:00  2407.66  2450.4500  2410.0000  2433.660  1087814
2021-06-18 00:00:00-04:00  2424.92  2432.0600  2400.3000  2400.620  1588611
2021-06-21 00:00:00-04:00  2411.09  2440.0000  2397.9700  2436.250   990946
2021-06-22 00:00:00-04:00  2435.77  2448.0000  2421.2800  2446.610   892332
2021-06-23 00:00:00-04:00  2443.50  2461.9100  2436.6900  2442.550   802706
2021-06-24 00:00:00-04:00  2453.60  2461.1106  2446.0000  2450.000   807016
2021-06-25 00:00:00-04:00  2441.63  2452.6600  2431.3050  2450.180   932717
2021-06-28 00:00:00-04:00  2451.00  2455.6950  2428.1100  2451.550  1119397
2021-06-29 00:00:00-04:00  2456.00  2460.4150  2433.8150  2444.890   826965
2021-06-30 00:00:00-04:00  2442.32  2445.1400  2427.0900  2441.389   916801
2021-07-01 00:00:00-04:00  2433.17  2451.7400  2430.6300  2448.890   753713
2021-07-02 00:00:00-04:00  2460.99  2508.0800  2461.1900  2505.680  1105587
2021-07-06 00:00:00-04:00  2512.97  2528.5600  2491.6501  2523.800  1093703
2021-07-07 00:00:00-04:00  2540.32  2545.6800  2523.0600  2529.470   992014
2021-07-08 00:00:00-04:00  2487.35  2511.9100  2480.4400  2501.850   992966

In [77]:
apple_df = pd.DataFrame()
apple_df["Google"] = apple_ticker["GOOGL"]["close"]
#apple_df = apple_df.pct_change().dropna()
apple_df.index = apple_df.index.date 
apple_df.tail()

,Google
2021-07-01,2448.89
2021-07-02,2505.68
2021-07-06,2523.80
2021-07-07,2529.47
2021-07-08,2501.85


In [78]:
apple_returns = apple_df.pct_change().dropna()
apple_returns.head(50)

,Google
2021-06-09,0.004124
2021-06-10,0.011342
2021-06-11,-0.001996
2021-06-14,0.007666
2021-06-15,-0.008069
2021-06-16,-0.005665
2021-06-17,0.007560
2021-06-18,-0.013576
2021-06-21,0.014842
2021-06-22,0.004252


In [79]:
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(start_date[:10], "%Y-%m-%d")
    past_date = datetime.strptime(end_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > past_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
        
    return all_headlines, all_dates

In [80]:
#inflation_headlines, dates = get_headlines("inflation")

In [81]:
advt_headlines, dates = get_headlines("advertising")

Fetching news about 'advertising'
******************************
retrieving news from: 2021-07-08 00:00:00
retrieving news from: 2021-07-07 00:00:00
retrieving news from: 2021-07-06 00:00:00
retrieving news from: 2021-07-05 00:00:00
retrieving news from: 2021-07-04 00:00:00
retrieving news from: 2021-07-03 00:00:00
retrieving news from: 2021-07-02 00:00:00
retrieving news from: 2021-07-01 00:00:00
retrieving news from: 2021-06-30 00:00:00
retrieving news from: 2021-06-29 00:00:00
retrieving news from: 2021-06-28 00:00:00
retrieving news from: 2021-06-27 00:00:00
retrieving news from: 2021-06-26 00:00:00
retrieving news from: 2021-06-25 00:00:00
retrieving news from: 2021-06-24 00:00:00
retrieving news from: 2021-06-23 00:00:00
retrieving news from: 2021-06-22 00:00:00
retrieving news from: 2021-06-21 00:00:00
retrieving news from: 2021-06-20 00:00:00
retrieving news from: 2021-06-19 00:00:00
retrieving news from: 2021-06-18 00:00:00
retrieving news from: 2021-06-17 00:00:00
retrieving 

In [82]:
#covid_headlines,_ = get_headlines("coronavirus")

In [83]:
sid = SentimentIntensityAnalyzer()

In [84]:
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [85]:
#inflation = headline_sentiment_summarizer_avg(inflation_headlines)
advertising = headline_sentiment_summarizer_avg(advt_headlines)
#regtech.to_csv("security.csv")
#covid = headline_sentiment_summarizer_avg(covid_headlines)

In [86]:
topic_sentiments = pd.DataFrame(
    {
        "advertising_avg": advertising,
    }
)
#"Inflation_avg": inflation,
#"Security_avg": regtech,
#"Covid_avg": covid,
print(topic_sentiments)
#topic_sentiments.to_csv("test.csv")

    advertising_avg
0         -0.032835
1         -0.139690
2          0.147860
3          0.049995
4          0.100415
5          0.009015
6          0.083680
7          0.130385
8         -0.037580
9          0.067540
10         0.031515
11         0.056470
12         0.088585
13         0.027995
14        -0.078795
15        -0.052875
16         0.069120
17         0.094835
18         0.109955
19         0.183010
20         0.010360
21         0.083395
22        -0.004190
23         0.033005
24         0.069805
25         0.088610
26         0.154360
27         0.013340
28         0.027290
29         0.016745


In [87]:
import datetime
topic_sentiments.index = pd.to_datetime(dates)
topic_sentiments = apple_returns.join(topic_sentiments).dropna(how="any")
#topic_sentiments.to_csv("topic_sentiment.csv")
display(topic_sentiments)

,Google,advertising_avg
2021-06-09,0.004124,0.016745
2021-06-10,0.011342,0.027290
2021-06-11,-0.001996,0.013340
2021-06-14,0.007666,0.069805
2021-06-15,-0.008069,0.033005
2021-06-16,-0.005665,-0.004190
2021-06-17,0.007560,0.083395
2021-06-18,-0.013576,0.010360
2021-06-21,0.014842,0.094835
2021-06-22,0.004252,0.069120


In [88]:
topic_sentiments.corr().style.background_gradient()

,Google,advertising_avg
Google,1.000000,0.413777
advertising_avg,0.413777,1.000000


In [89]:
topic_sentiments.hvplot.line(figsize=(20,10))

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [90]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [91]:
window_size = 2
feature_column = 1
target_column = 0
X, y = window_data(topic_sentiments, window_size, feature_column, target_column)
#topic_sentiments.head()

In [92]:
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X[split :]
y_train = y[: split]
y_test = y[split :]

In [93]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [94]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print(f"X_train sample values:\n{X_train[:5]}\n")
print(f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.45673583]
  [0.46804615]]

 [[0.50714695]
  [0.40649886]]

 [[0.44045798]
  [0.65562198]]

 [[0.71039296]
  [0.49326068]]

 [[0.53446792]
  [0.32915665]]]

X_test sample values:
[[[0.51051726]
  [0.48668681]]

 [[0.52734487]
  [0.64562882]]

 [[0.69956497]
  [0.18184024]]

 [[0.19703126]
  [0.92290044]]

 [[1.        ]
  [0.71683837]]]


In [95]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [96]:
model = Sequential()
number_units = 30 
dropout_fraction = .2

#Layer1
model.add(LSTM(units=number_units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(dropout_fraction))
#Layer2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
#Layer3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
#Output Layer
model.add(Dense(1))

In [97]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [98]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 2, 30)             3840      
_________________________________________________________________
dropout_6 (Dropout)          (None, 2, 30)             0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 2, 30)             7320      
_________________________________________________________________
dropout_7 (Dropout)          (None, 2, 30)             0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_8 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [99]:
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
12/12 [==============================] - 3s 3ms/step - loss: 0.1686
Epoch 2/10
12/12 [==============================] - 0s 3ms/step - loss: 0.1294
Epoch 3/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0867
Epoch 4/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0514
Epoch 5/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0480
Epoch 6/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0380
Epoch 7/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0402
Epoch 8/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0436
Epoch 9/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0395
Epoch 10/10
12/12 [==============================] - 0s 3ms/step - loss: 0.0503


In [100]:
model.evaluate(X_test, y_test)  

1/1 [==============================] - 1s 887ms/step - loss: 0.0699


0.06990326195955276

In [101]:
predicted = model.predict(X_test)

In [102]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [103]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = topic_sentiments.index[-len(real_prices): ]) 
stocks.head(100)

,Real,Predicted
2021-06-30,-0.002717,0.000877
2021-07-01,-0.001432,0.001185
2021-07-02,0.003072,0.001231
2021-07-06,0.023190,0.000204
2021-07-07,0.007232,0.003341
2021-07-08,0.002247,0.002792


In [104]:
stocks.hvplot.line(label = "Predicting Google's Return Using NLP - (2Day Window)",xlabel = "Date", ylabel= "Return" )

:NdOverlay   [Variable]
   :Curve   [index]   (value)